In [1]:
import pandas as pd
import tqdm.notebook
import random
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics

In [2]:
from models.supervised import CRSum
from UniversalSentenceEncoder import USEEmbedder
from Preprocessors import CRSumPreprocessor

In [3]:
test_data = pd.read_csv("Topic_Data.csv")

In [4]:
summarizer = CRSum(embedding_model=None, preprocessor=CRSumPreprocessor, M=5, N=5, verbose=False)

In [5]:
summarizer.loadWeights("best_model.h5")

In [6]:
embedder = USEEmbedder(None)

In [7]:
test_data.head()

,lead,body,Language,Cluster
0,Mit drastischen Worten warnen Politiker vor de...,"Angela Merkel hat schon viele Krisen erlebt, a...",German,Climate
1,In immer mehr Gebieten des Great Barrier Reefs...,Geht die Farbenpracht des Great Barrier Reefs ...,German,Climate
2,Eine Stunde das Licht ausschalten als Zeichen ...,"Die Earth Hour, die ""Stunde der Erde"", soll mi...",German,Climate
3,Die nächsten zehn Jahre werden noch heißer und...,Der Deutsche Wetterdienst zog am Dienstag in s...,German,Climate
4,Der Klimawandel lässt nach Prognosen des Deuts...,Das laufende Jahr könnte nach den Angaben des ...,German,Climate


In [8]:
summaries = []

In [9]:
for i, row in tqdm.notebook.tqdm(test_data.iterrows(), total=len(test_data.index)):
    try:
        smry = summarizer.summarize(row.body, row.Language, 0.2)
    except:
        smry = " "
    summaries.append(smry)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/swrdata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/swrdata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/swrdata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
test_data["Pred_Summary"] = summaries

In [11]:
test_data.head()

,lead,body,Language,Cluster,Pred_Summary
0,Mit drastischen Worten warnen Politiker vor de...,"Angela Merkel hat schon viele Krisen erlebt, a...",German,Climate,"Angela Merkel hat schon viele Krisen erlebt, a..."
1,In immer mehr Gebieten des Great Barrier Reefs...,Geht die Farbenpracht des Great Barrier Reefs ...,German,Climate,Denn das weltgrößte Riff vor der Nordostküste ...
2,Eine Stunde das Licht ausschalten als Zeichen ...,"Die Earth Hour, die ""Stunde der Erde"", soll mi...",German,Climate,"Die Earth Hour, die ""Stunde der Erde"", soll mi..."
3,Die nächsten zehn Jahre werden noch heißer und...,Der Deutsche Wetterdienst zog am Dienstag in s...,German,Climate,Die Niederschläge werden in diesem Jahr vermut...
4,Der Klimawandel lässt nach Prognosen des Deuts...,Das laufende Jahr könnte nach den Angaben des ...,German,Climate,Das laufende Jahr könnte nach den Angaben des ...


In [12]:
sum_embedding_matrix = np.zeros((0,512))
body_embedding_matrix = np.zeros((0,512))

In [13]:
for _, row in tqdm.notebook.tqdm(test_data.iterrows(), total = len(test_data.index)):
    sum_embedding_matrix = np.concatenate((sum_embedding_matrix, embedder.embed(row.Pred_Summary)), axis=0)
    body_embedding_matrix = np.concatenate((body_embedding_matrix, embedder.embed(row.body)), axis=0)

In [14]:
test_data.Cluster.unique()

array(['Climate', 'Coronavirus', 'Korea', 'Oilprice'], dtype=object)

In [15]:
test_data["Topic_Cat"] = pd.factorize(test_data.Cluster)[0]

In [16]:
test_data.head()

,lead,body,Language,Cluster,Pred_Summary,Topic_Cat
0,Mit drastischen Worten warnen Politiker vor de...,"Angela Merkel hat schon viele Krisen erlebt, a...",German,Climate,"Angela Merkel hat schon viele Krisen erlebt, a...",0
1,In immer mehr Gebieten des Great Barrier Reefs...,Geht die Farbenpracht des Great Barrier Reefs ...,German,Climate,Denn das weltgrößte Riff vor der Nordostküste ...,0
2,Eine Stunde das Licht ausschalten als Zeichen ...,"Die Earth Hour, die ""Stunde der Erde"", soll mi...",German,Climate,"Die Earth Hour, die ""Stunde der Erde"", soll mi...",0
3,Die nächsten zehn Jahre werden noch heißer und...,Der Deutsche Wetterdienst zog am Dienstag in s...,German,Climate,Die Niederschläge werden in diesem Jahr vermut...,0
4,Der Klimawandel lässt nach Prognosen des Deuts...,Das laufende Jahr könnte nach den Angaben des ...,German,Climate,Das laufende Jahr könnte nach den Angaben des ...,0


In [17]:
sil = []
ks = []
kmax = 20

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in tqdm.notebook.tqdm(range(2, kmax+1)):
    kmeans = KMeans(n_clusters = k, n_jobs=-1).fit(sum_embedding_matrix)
    labels = kmeans.labels_
    sil.append(metrics.silhouette_score(sum_embedding_matrix, labels, metric = 'euclidean'))
    ks.append(k)

In [18]:
ks[sil.index(max(sil))]

7

In [19]:
sum_clustering = KMeans(n_clusters = ks[sil.index(max(sil))], n_jobs=-1).fit(sum_embedding_matrix)

In [20]:
body_clustering = KMeans(n_clusters = ks[sil.index(max(sil))], n_jobs=-1).fit(body_embedding_matrix)

In [21]:
sum_pred_clusters  = sum_clustering.labels_.tolist()
body_pred_clusters  = body_clustering.labels_.tolist()
true_clusters = test_data.Topic_Cat.tolist()

In [22]:
len(sum_pred_clusters) == len(true_clusters)# == len(body_pred_clusters)

True

In [23]:
metrics.adjusted_mutual_info_score(true_clusters, sum_pred_clusters)  

0.5465393013840192

In [24]:
metrics.adjusted_mutual_info_score(true_clusters, body_pred_clusters)  

0.5019665888543414

## Random Baseline

In [25]:
random_clusters = [np.random.choice(np.arange(0, 4), p=[0.2, 0.32, 0.22, 0.26]) for i in range(len(true_clusters))]

In [26]:
metrics.adjusted_mutual_info_score(true_clusters, random_clusters)  

-0.0005609384961465434

## Majority Class Baseline

In [27]:
majority_clusters = [1]*len(true_clusters)

In [28]:
metrics.adjusted_mutual_info_score(true_clusters, majority_clusters)  

-1.4158281625007905e-15